<a href="https://colab.research.google.com/github/Tongxi-Hu/deep-learning/blob/main/d2l/6_1_convolutional_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6-1 2d-convolution

###6-1-1 construction convolutional layer

In [ ]:
!pip3 install d2l
import torch
from torch import nn
from d2l import torch as d2l

In [6]:
def corr2d(X,K):
    """Compute 2D cross-correlation."""
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [7]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [8]:
#convolutional layer
class Conv2D(nn.Module):
    def __init__(self,kernel_size) -> None:
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zero(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [9]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [11]:
K=torch.tensor([[1.0,-1.0]])
#if horizontally adjacent elements are the same, the output is 0,
#Otherwise, the output is non-zero.
#we detect 1 for the edge from white to black and -1 for the edge from black to white. 
#All other outputs take value 0.
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [12]:
corr2d(X.t(), K)
#The kernel K only detects vertical edges.

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [16]:
#Learning a Kernel
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X=X.reshape(1,1,6,8)
Y=Y.reshape(1,1,6,7)
for i in range(10):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=3e-2*conv2d.weight.grad
    if(i+1)%2==0:
        print(f'batch{i+1}, loss{l.sum():.3f}')
print(conv2d.weight.data.reshape((1,2)))

batch2, loss2.238
batch4, loss0.397
batch6, loss0.075
batch8, loss0.016
batch10, loss0.004
tensor([[ 0.9975, -0.9872]])


###6-1-2 Padding and Stride

In [17]:
import torch 
from torch import nn
#padding to preserve size
def comp_conv2d(cov2d,X):
    X=X.reshape((1,1)+X.shape)
    Y=cov2d(X)
    return Y.reshape(Y.shape[2:])
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1)
X=torch.rand((8,8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [19]:
# Here, we use a convolution kernel with a height of 5 and a width of 3. The
# padding numbers on either side of the height and width are 2 and 1,
# respectively
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [18]:
#stride to shrink size
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [20]:
conv2d=nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])